In [1]:
#importing important libraries:
#import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
#loading train data:
data=pd.read_csv('flight_delays_train.csv')
data

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [3]:
#cleaning data:
data.isnull().values.any()

False

In [4]:
#creating training examples and label:
y=data['dep_delayed_15min'].values
X=data.drop(['dep_delayed_15min'],axis=1)
X

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423
...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140


In [5]:
y=y.reshape(100000,1);

In [6]:
#converting string values to integer:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

In [7]:
#converted Yes No to 1,0:
y=le.fit_transform(y)
#np.set_printoptions(threshold=sys.maxsize)


C:\Users\Pushkaraj\anaconda3\envs\puspa\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
#converted strings to unique integer:
x1=le.fit_transform(data['Month'].values)
x1=x1/np.average(x1)
x2=le.fit_transform(data['DayofMonth'].values)
x2=x2/np.average(x2)
x3=le.fit_transform(data['DayOfWeek'].values)
x3=x3/np.average(x3)
x4=le.fit_transform(data['DepTime'].values)
x4=x4/np.average(x4)
x5=le.fit_transform(data['UniqueCarrier'].values)
x5=x5/np.average(x5)
x6=le.fit_transform(data['Origin'].values)
x6=x6/np.average(x6)
x7=le.fit_transform(data['Dest'].values)
x7=x7/np.average(x7)
x8=le.fit_transform(data['Distance'].values)
x8=x1/np.average(x8)
X=np.array([x1,x2,x3,x4,x5,x6,x7,x8]).T


In [54]:
X.shape

(100000, 8)

In [55]:
y.shape

(100000,)

In [56]:
X

array([[1.79787886e+00, 8.76961020e-01, 2.03263738e+00, ...,
        1.26424195e-01, 5.53030248e-01, 3.59703489e-03],
       [1.07872732e+00, 8.09502480e-01, 6.77545794e-01, ...,
        1.52411391e+00, 1.21241247e+00, 2.15822093e-03],
       [1.97766675e+00, 7.42043940e-01, 1.35509159e+00, ...,
        1.60137314e+00, 4.18317751e-01, 3.95673838e-03],
       ...,
       [0.00000000e+00, 1.07933664e+00, 3.38772897e-01, ...,
        5.97003143e-01, 9.28807211e-01, 0.00000000e+00],
       [1.07872732e+00, 1.28171226e+00, 1.01631869e+00, ...,
        5.54861744e-01, 7.58644058e-01, 2.15822093e-03],
       [3.59575773e-01, 5.39668320e-01, 1.01631869e+00, ...,
        1.72077376e+00, 1.79380324e+00, 7.19406978e-04]])

In [57]:
#selection of best 5 features:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_new = SelectKBest(chi2, k=5).fit_transform(X, y)

In [58]:
X_new.shape

(100000, 5)

In [59]:
#defining sigmoid function: this function calculates sigmoid of numpy array z
def sigmoid(z):
    s=1/(1+np.exp(-z))
    return s

In [60]:
#function initializing parameters of neural network to zero:
def initialize_to_zeros(dim):
    w=np.zeros((dim,1))
    b=0.0
    return w,b

In [61]:
#forward and backward propagation to calculate output and derivatives:
def propagate(w,b,X,Y):
    m = X.shape[1]
    #fwd propagation:
    A = sigmoid(np.dot(w.T,X.T)+b)    #compute activation
    cost = -1 / m * np.sum(Y*np.log(A)+(1-Y)*np.log(1-A), axis = 1, keepdims = True)     # compute cost
    #backward propagation:
    dw = 1 / m * np.dot(X.T,(A-Y).T)
    db = 1 / m * np.sum(A-Y)
    cost = np.squeeze(cost)
    gradients={"dw": dw,
               "db": db}
    return gradients,cost

In [62]:
#function for optizing parameters using gradient descent algorithm:
def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009):
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w=w, b=b, X=X, Y=y)
        dw=grads["dw"]
        db=grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads    

In [63]:
#function to predict the label is 0 or 1:
def predict(w,b,X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    A=sigmoid(np.dot(w.T,X)+b)
    
    for i in range(A.shape[1]):
        if A[0,i] > 0.5:
            Y_prediction[0,i]=1
        else:
            Y_prediction[0,i]=0
    return Y_prediction

In [64]:
#using all above functions for prediction:
w, b = initialize_to_zeros(X_new.shape[1])
grads, cost = propagate(w,b,X_new,y)
grads

{'dw': array([[6172.67547747],
        [6230.11996827],
        [6135.88858437],
        [5357.11505891],
        [6250.84133546]]),
 'db': 6191.200000000001}

In [65]:

X_new

array([[1.79787886, 0.87696102, 2.03263738, 1.54004359, 0.12642419],
       [1.07872732, 0.80950248, 0.67754579, 1.19781168, 1.52411391],
       [1.97766675, 0.74204394, 1.35509159, 1.06758184, 1.60137314],
       ...,
       [0.        , 1.07933664, 0.3387729 , 1.49007167, 0.59700314],
       [1.07872732, 1.28171226, 1.01631869, 1.14783976, 0.55486174],
       [0.35957577, 0.53966832, 1.01631869, 1.39769934, 1.72077376]])

In [66]:
params,grads=optimize(w, b, X_new, y, num_iterations=100, learning_rate=0.009)


C:\Users\Pushkaraj\AppData\Local\Temp\ipykernel_1224\1032241046.py:6: RuntimeWarning: divide by zero encountered in log
  cost = -1 / m * np.sum(Y*np.log(A)+(1-Y)*np.log(1-A), axis = 1, keepdims = True)     # compute cost
C:\Users\Pushkaraj\AppData\Local\Temp\ipykernel_1224\1032241046.py:6: RuntimeWarning: invalid value encountered in multiply
  cost = -1 / m * np.sum(Y*np.log(A)+(1-Y)*np.log(1-A), axis = 1, keepdims = True)     # compute cost
C:\Users\Pushkaraj\AppData\Local\Temp\ipykernel_1224\1440447879.py:3: RuntimeWarning: overflow encountered in exp
  s=1/(1+np.exp(-z))


In [67]:
params

{'w': array([[-47.63131696],
        [-52.66373804],
        [-45.29808476],
        [ 81.80323185],
        [-52.10912349]]),
 'b': -141.72975936850736}

In [68]:
grads

{'dw': array([[3890.11917235],
        [3782.64342281],
        [3856.21662704],
        [6984.77058755],
        [3717.80966427]]),
 'db': 4339.21063663802}

In [69]:
w=params['w']
b=params['b']


In [70]:
Y_pred=predict(w,b,X_new.T)

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y, Y_pred.T)

0.80956

In [47]:
testdata=pd.read_csv('flight_delays_test.csv')
testdata

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258
...,...,...,...,...,...,...,...,...
99995,c-6,c-5,c-2,852,WN,CRP,HOU,187
99996,c-11,c-24,c-6,1446,UA,ORD,LAS,1515
99997,c-1,c-30,c-2,1509,OO,ORD,SGF,438
99998,c-1,c-5,c-5,804,DL,LGA,ATL,761


In [50]:
test=testdata[['DayofMonth','DayOfWeek','DepTime','Origin','Distance']].values
test

array([['c-25', 'c-3', 615, 'MRY', 598],
       ['c-17', 'c-2', 739, 'LAS', 1235],
       ['c-2', 'c-7', 651, 'GSP', 577],
       ...,
       ['c-30', 'c-2', 1509, 'ORD', 438],
       ['c-5', 'c-5', 804, 'LGA', 761],
       ['c-29', 'c-1', 834, 'MKE', 297]], dtype=object)

In [51]:
x1_test=le.fit_transform(data['DayofMonth'].values)
x1_test=x1_test/np.average(x1_test)
x2_test=le.fit_transform(data['DayOfWeek'].values)
x2_test=x2_test/np.average(x2_test)
x3_test=le.fit_transform(data['DepTime'].values)
x3_test=x3_test/np.average(x3_test)
x4_test=le.fit_transform(data['Origin'].values)
x4_test=x4_test/np.average(x4_test)
x5_test=le.fit_transform(data['Distance'].values)
x5_test=x5_test/np.average(x5_test)
X_test=np.array([x1_test,x2_test,x3_test,x4_test,x5_test]).T
X_test

array([[0.87696102, 2.03263738, 1.54004359, 0.12642419, 1.15440988],
       [0.80950248, 0.67754579, 1.19781168, 1.52411391, 1.30246245],
       [0.74204394, 1.35509159, 1.06758184, 1.60137314, 0.61221737],
       ...,
       [1.07933664, 0.3387729 , 1.49007167, 0.59700314, 1.65658819],
       [1.28171226, 1.01631869, 1.14783976, 0.55486174, 0.11003907],
       [0.53966832, 1.01631869, 1.39769934, 1.72077376, 0.93833316]])

In [52]:
Y_pred=predict(w,b,X_test.T)  #prediction on test